# Study model Wine Quality

## Data pre-processing

In [2]:
# load data
import polars as pl
df_csv = pl.read_csv("../data/Wines.csv")
# df_csv

In [3]:
df_csv.describe()

statistic,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0,1143.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,0.99673,3.311015,0.657708,10.442111,5.657043,804.969379
"""std""",1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.78213,0.001925,0.156664,0.170399,1.082196,0.805824,463.997116
"""min""",4.6,0.12,0.0,0.9,0.012,1.0,6.0,0.99007,2.74,0.33,8.4,3.0,0.0
"""25%""",7.1,0.395,0.09,1.9,0.07,7.0,21.0,0.99557,3.21,0.55,9.5,5.0,412.0
"""50%""",7.9,0.52,0.25,2.2,0.079,13.0,37.0,0.99668,3.31,0.62,10.2,6.0,794.0
"""75%""",9.1,0.64,0.42,2.6,0.09,21.0,61.0,0.99785,3.4,0.73,11.1,6.0,1210.0
"""max""",15.9,1.58,1.0,15.5,0.611,68.0,289.0,1.00369,4.01,2.0,14.9,8.0,1597.0


In [4]:
# Définir `X` comme toutes les colonnes sauf "id" et "quality"
X = df_csv.select([col for col in df_csv.columns if col not in ["Id", "quality"]])
# Définir `y` comme la colonne "quality"
y = df_csv.select("quality")

# # Afficher X et y pour vérification
# print("X (features) :\n", X)
# print("\nY (target) :\n", y)

In [5]:
# Afficher les valeurs différentes de quality (on a remarqué qu'il en manque certaines)
# for col in y.columns:
#     valeurs_uniques = y.select(pl.col(col).unique()).to_series()
#     print(f"Valeurs uniques pour la colonne '{col}':\n", valeurs_uniques)
#     print("\n" + "-"*50 + "\n")

Dans la colonne quality il manque 1, 2, 9 et 10. D'où le choix d'une regression et non une classification.

## Model SVM (regression)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(914, 11) (229, 11) (914, 1) (229, 1)


In [7]:
import numpy as np

def get_accuracy(y_test, y_test_hat):
    """ Retourne l'accuracy du modèle """
    # Arrondir les prédictions à l'entier le plus proche
    y_test_hat_rounded = np.round(y_test_hat)
    # Comparer les prédictions arrondies avec les valeurs réelles
    correct_predictions = np.sum(y_test_hat_rounded == y_test)
    # Calculer l'accuracy (le pourcentage de bonnes prédictions)
    accuracy = correct_predictions / len(y_test)
    return accuracy

### Linear

In [ ]:
svm_linear = svm.SVR(kernel='linear')
svm_linear.fit(X_train, y_train)
y_test_hat_linear = svm_linear.predict(X_test)

# # DEBUG
# print("Real, Predict")
# for i in range(len(y_test)):
#     print(f'{y_test[i].item()}, {np.round(y_test_hat_linear[i])}')
# # FIN DEBUG

mse_linear = mean_squared_error(y_test_hat_linear, y_test)
print(f'Linear SVM achieved {round(mse_linear, 3)} MSE.')
print(f'accuracy = {get_accuracy(y_test, y_test_hat_linear)}')

/home/patrice/Documents/Etudes/2024-2025_CY-Tech_Ing3/Cours/Architecture_Microservices/FastAPI/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Linear SVM achieved 0.369 MSE.
accuracy = 92.43668122270742


### Poly (BEST)

In [ ]:
svm_poly = svm.SVR(kernel='poly')
svm_poly.fit(X_train, y_train)
y_test_hat_poly = svm_poly.predict(X_test)

# # DEBUG
# print("Real, Predict")
# for i in range(len(y_test)):
#     print(f'{y_test[i].item()}, {np.round(y_test_hat_linear[i])}')
# # FIN DEBUG

mse_poly = mean_squared_error(y_test_hat_poly, y_test)
print(f'Polynomial SVM achieved {round(mse_poly, 3)} MSE.')
print(f'accuracy = {get_accuracy(y_test, y_test_hat_poly)}')

Polynomial SVM achieved 0.494 MSE.
accuracy = 98.117903930131


/home/patrice/Documents/Etudes/2024-2025_CY-Tech_Ing3/Cours/Architecture_Microservices/FastAPI/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
# Save model
import joblib
joblib.dump(svm_poly, "../models/model_wine_recommendation.pkl")

['../models/model_wine_recommendation.pkl']

In [15]:
# Load and use model
model = joblib.load("../models/model_wine_recommendation.pkl")

# Test
y_test_hat_poly = model.predict(X_test)
mse_poly = mean_squared_error(y_test_hat_poly, y_test)
print(f'Polynomial SVM achieved {round(mse_poly, 3)} MSE.')
print(f'accuracy = {get_accuracy(y_test, y_test_hat_poly)}')

Polynomial SVM achieved 0.494 MSE.
accuracy = 98.117903930131


### Rbf

In [10]:
svm_rbf = svm.SVR(kernel='rbf')
svm_rbf.fit(X_train, y_train)
y_test_hat_rbf = svm_rbf.predict(X_test)
mse_rbf = mean_squared_error(y_test_hat_rbf, y_test)
print(f'RBF SVM achieved {round(mse_rbf, 3)} MSE.')
print(f'accuracy = {get_accuracy(y_test, y_test_hat_rbf)}')

RBF SVM achieved 0.423 MSE.
accuracy = 97.9475982532751


/home/patrice/Documents/Etudes/2024-2025_CY-Tech_Ing3/Cours/Architecture_Microservices/FastAPI/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
